In [ ]:
hydraA_coord = SkyCoord(ra=139.5235458*u.deg, dec=-12.0955528*u.deg)
target = FixedTarget(coord=sirius_coord, name="Sirius")

In [ ]:
from numpy import deg2rad, sin, cos, tan, arctan, fabs
import numpy as np
import datetime
import collections

import math
PI = math.pi
SMALL_FLOAT = 1.0e-12


def Julian(dobj):
    """Returns the number of julian days for the specified date-time.
    Input = datetime object.
    """

    if isinstance(dobj, datetime.datetime):
        dobj = np.array([dobj], 'datetime64[us]')
    elif isinstance(dobj, np.ndarray):
        dobj = dobj.astype('datetime64[us]')
    elif isinstance(dobj, collections.Sequence):
        dobj = np.array(dobj, 'datetime64[us]')

    day = (
        dobj - dobj.astype('datetime64[M]')).astype('f') / (24. * 3600. * 1000000) + 1.0
    month = (
        dobj.astype('datetime64[M]') - dobj.astype('datetime64[Y]')).astype('f') + 1
    year = dobj.astype('datetime64[Y]').astype('f') + 1970

    year = np.where(np.less(month, 3), year - 1, year)
    month = np.where(np.less(month, 3), month + 12, month)

    cond1 = np.greater(year, 1582)
    cond2 = np.logical_and(np.equal(year, 1582), np.greater(month, 10))
    cond3 = np.logical_and(np.logical_and(np.equal(year, 1582), np.equal(month, 10)),
                           np.greater(day, 15))

    a__ = np.divide(year, 100).astype('i')
    b__ = np.where(np.logical_or(np.logical_or(cond1, cond2), cond3),
                   2 - a__ + a__ / 4, 0)

    c__ = (365.25 * year).astype('i')
    e__ = (30.6001 * (month + 1)).astype('i')
    return b__ + c__ + e__ + day + 1720994.5






def moon_position(jday, lsun):
    """Get the moon position"""

    if isinstance(jday, collections.Sequence):
        jday = np.array(jday, 'float64')
    elif not isinstance(jday, np.ndarray):
        jday = np.array([jday], 'float64')

    if isinstance(lsun, collections.Sequence):
        lsun = np.array(lsun, 'float64')
    elif not isinstance(lsun, np.ndarray):
        lsun = np.array([lsun], 'float64')

    ms_ = 0.985647332099 * jday - 3.762863
    ms_ = np.where(np.less(ms_, 0), ms_ + 360.0, ms_)

    mpos = 13.176396 * jday + 64.975464
    i__ = np.divide(mpos, 360.0).astype('i')
    mpos = mpos - i__ * 360.0
    mpos = np.where(np.less(mpos, 0), mpos + 360.0, mpos)

    mm_ = mpos - 0.1114041 * jday - 349.383063
    i__ = np.divide(mm_, 360.0).astype('i')
    mm_ = mm_ - i__ * 360.0

    ev_ = 1.2739 * sin(deg2rad(2 * (mpos - lsun) - mm_))
    sms = sin(deg2rad(ms_))
    ae_ = 0.1858 * sms
    mm_ = mm_ + ev_ - ae_ - 0.37 * sms
    ec_ = 6.2886 * sin(deg2rad(mm_))
    mpos = mpos + ev_ + ec_ - ae_ + 0.214 * sin(deg2rad(2 * mm_))
    mpos = 0.6583 * sin(deg2rad(2 * (mpos - lsun))) + mpos

    if isinstance(mpos, np.ndarray) and len(mpos) == 1:
        return mpos[0]
    else:
        return mpos